Jotta voit ajaa seuraavat muistikirjat, sinun täytyy, jos et ole vielä tehnyt niin, ottaa käyttöön malli, joka käyttää `text-embedding-ada-002`-pohjamallia ja asettaa käyttöönoton nimen .env-tiedostoon nimellä `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Seuraavaksi aiomme ladata upotusindeksin Pandas Dataframeen. Upotusindeksi on tallennettu JSON-tiedostoon nimeltä `embedding_index_3m.json`. Upotusindeksi sisältää upotukset jokaiselle YouTube-transkriptille aina lokakuun 2023 loppuun asti.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Seuraavaksi luomme funktion nimeltä `get_videos`, joka etsii kyselyä Embedding Indexistä. Funktio palauttaa viisi videota, jotka ovat kyselyyn nähden kaikkein samankaltaisimpia. Funktio toimii seuraavasti:

1. Ensin luodaan kopio Embedding Indexistä.
2. Seuraavaksi kyselyn Embedding lasketaan OpenAI Embedding API:n avulla.
3. Sitten Embedding Indexiin lisätään uusi sarake nimeltä `similarity`. `similarity`-sarake sisältää kosinietäisyyden kyselyn Embeddingin ja jokaisen videosegmentin Embeddingin välillä.
4. Tämän jälkeen Embedding Index suodatetaan `similarity`-sarakkeen perusteella. Mukaan otetaan vain ne videot, joiden kosinietäisyys on vähintään 0.75.
5. Lopuksi Embedding Index järjestetään `similarity`-sarakkeen mukaan ja palautetaan viisi parasta videota.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Tämä funktio on hyvin yksinkertainen, se vain tulostaa hakukyselyn tulokset.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Ensin Embedding Index ladataan Pandas Dataframeen.
2. Seuraavaksi käyttäjää pyydetään syöttämään kysely.
3. Sitten kutsutaan `get_videos`-funktiota, joka etsii kyselyä Embedding Indexistä.
4. Lopuksi kutsutaan `display_results`-funktiota, joka näyttää tulokset käyttäjälle.
5. Käyttäjää pyydetään syöttämään uusi kysely. Prosessi jatkuu, kunnes käyttäjä syöttää `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.fi.png)

Sinua pyydetään syöttämään kysely. Kirjoita kysely ja paina enter. Sovellus palauttaa listan videoista, jotka liittyvät kyselyyn. Sovellus antaa myös linkin kohtaan videossa, jossa kysymykseen vastataan.

Tässä muutamia esimerkkikyselyitä, joita voit kokeilla:

- Mitä on Azure Machine Learning?
- Miten konvoluutioneuroverkot toimivat?
- Mikä on neuroverkko?
- Voinko käyttää Jupyter Notebookeja Azure Machine Learningin kanssa?
- Mitä on ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisissä tapauksissa suosittelemme ammattimaisen ihmiskääntäjän käyttöä. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkinnoista.
